In [1]:
#pip install -r requirements.txt

In [2]:
import openai
import os
import pandas as pd
import numpy as np
import json
import tiktoken
import psycopg2
import ast
import pgvector
import math
from psycopg2.extras import execute_values
from pgvector.psycopg2 import register_vector

In [3]:
import os
import json

with open('secrets.json') as secrets_file:
    secrets = json.load(secrets_file)
    openai.api_key = secrets['OPENAI_API_KEY']


In [4]:
# Load your CSV file into a pandas DataFrame
df = pd.read_csv('movies-dataset.csv')
df.head()

,title,image,plot
0,'68 (film),https://upload.wikimedia.org/wikipedia/en/e/e5...,The father escaped the Soviet invasion of Buda...
1,'Gator Bait II: Cajun Justice,https://upload.wikimedia.org/wikipedia/en/9/91...,When a sweet city girl is initiated into the r...
2,"'night, Mother (film)",https://upload.wikimedia.org/wikipedia/en/e/e7...,Jessie is a middle-aged woman living with her ...
3,...All the Marbles,https://upload.wikimedia.org/wikipedia/en/3/35...,Harry is the manager of a tag team of attracti...
4,10 to Midnight,https://upload.wikimedia.org/wikipedia/en/0/04...,Warren Stacey (Gene Davis) is a young office e...


In [5]:
# Helper functions to help us create the embeddings

# Helper func: calculate number of tokens
def num_tokens_from_string(string: str, encoding_name = "cl100k_base") -> int:
    if not string:
        return 0
    # Returns the number of tokens in a text string
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

# Helper function: calculate length of essay
def get_essay_length(essay):
    word_list = essay.split()
    num_words = len(word_list)
    return num_words

# Helper function: calculate cost of embedding num_tokens
# Assumes we're using the text-embedding-ada-002 model
# See https://openai.com/pricing
def get_embedding_cost(num_tokens):
    return num_tokens/1000*0.0001
# Helper function: calculate total cost of embedding all plot in the dataframe
def get_total_embeddings_cost():
    total_tokens = 0
    for i in range(len(df.index)):
        text = df['plot'][i]
        token_len = num_tokens_from_string(text)
        total_tokens = total_tokens + token_len
    total_cost = get_embedding_cost(total_tokens)
    return total_cost

# Helper function: get embeddings for a text
def get_embeddings(text):
    response = openai.Embedding.create(
        model="text-embedding-ada-002",
        input = text.replace("\n"," ")
    )
    embedding = response['data'][0]['embedding']
    return embedding

In [6]:

# Create new list with small plot chunks to not hit max token limits
# Note: the maximum number of tokens for a single request is 8191
# https://openai.com/docs/api-reference/requests

# list for chunked plot and embeddings
new_list = []
# Split up the text into token sizes of around 512 tokens
new_list = []

for i in range(len(df.index)):
    text = str(df['plot'][i])
    token_len = num_tokens_from_string(text)
    if token_len <= 512:
        new_list.append([df['title'][i], df['image'][i], df['plot'][i], token_len])
    else:
        # add plot to the new list in chunks
        start = 0
        ideal_token_size = 512
        # 1 token ~ 3/4 of a word
        ideal_size = int(ideal_token_size // (4/3))
        end = ideal_size
        #split text by spaces into words
        words = text.split()

        #remove empty spaces
        words = [x for x in words if x != ' ']

        total_words = len(words)
        
        #calculate iterations
        chunks = total_words // ideal_size
        if total_words % ideal_size != 0:
            chunks += 1
        
        new_plot = []
        for j in range(chunks):
            if end > total_words:
                end = total_words
            new_plot = words[start:end]
            new_plot_string = ' '.join(new_plot)
            new_plot_token_len = num_tokens_from_string(new_plot_string)
            if new_plot_token_len > 0:
                new_list.append([df['title'][i], new_plot_string, df['image'][i], new_plot_token_len])
            start += ideal_size
            end += ideal_size

In [7]:
# Create embeddings for each piece of plot
for i in range(len(new_list)):
    text = new_list[i][1]
    embedding = get_embeddings(text)
    new_list[i].append(embedding)

# Create a new dataframe from the list
df_new = pd.DataFrame(new_list, columns=['title', 'image', 'plot', 'tokens', 'embeddings'])
df_new.head()

,title,image,plot,tokens,embeddings
0,'68 (film),https://upload.wikimedia.org/wikipedia/en/e/e5...,The father escaped the Soviet invasion of Buda...,162,"[-0.012753772549331188, -0.015340202488005161,..."
1,'Gator Bait II: Cajun Justice,https://upload.wikimedia.org/wikipedia/en/9/91...,When a sweet city girl is initiated into the r...,69,"[-0.023718098178505898, -0.002384522929787636,..."
2,"'night, Mother (film)",https://upload.wikimedia.org/wikipedia/en/e/e7...,Jessie is a middle-aged woman living with her ...,94,"[0.0032459578942507505, -0.012125869281589985,..."
3,...All the Marbles,https://upload.wikimedia.org/wikipedia/en/3/35...,Harry is the manager of a tag team of attracti...,64,"[-0.034039661288261414, 0.001437551574781537, ..."
4,10 to Midnight,Warren Stacey (Gene Davis) is a young office e...,https://upload.wikimedia.org/wikipedia/en/0/04...,498,"[-0.008637497201561928, -0.006688959430903196,..."


In [8]:
# Save the dataframe with embeddings as a CSV file
df_new.to_csv('movies-datase-and-embeddings.csv', index=False)
# It may also be useful to save as a json file, but we won't use this in the tutorial
#df_new.to_json('blog_data_and_embeddings.json')

In [9]:
# Conection for PostgreSQL database connection string
with open('secrets.json') as secrets_file:
    secrets = json.load(secrets_file)
    connection_string = secrets['POSTGRESQL_CONNECTION_STRING_V2']

In [10]:
# Connect to PostgreSQL database in MYDB using connection string
conn = psycopg2.connect(connection_string)
cur = conn.cursor()
#install pgvector
cur.execute("CREATE EXTENSION IF NOT EXISTS vector");
conn.commit()

In [11]:
# Register the vector type with psycopg2

register_vector(conn)

In [12]:
# Create table to store embeddings and metadata

table_create_command = """
CREATE TABLE IF NOT EXISTS embeddings (
            id bigserial primary key, 
            title text,
            plot text,
            image text,
            tokens integer,
            embedding vector(1536)
            );
            """
cur.execute(table_create_command)
cur.close()
conn.commit()


In [14]:
#Batch insert embeddings and metadata from dataframe into PostgreSQL database
register_vector(conn)
cur = conn.cursor()
# Prepare the list of tuples to insert
data_list = [(row['title'], row['plot'], row['image'], int(row['tokens']), np.array(row['embeddings'])) for index, row in df_new.iterrows()]
# Use execute_values to perform batch insertion
execute_values(cur, "INSERT INTO embeddings (title, plot, image, tokens, embedding) VALUES %s", data_list)
# Commit after we insert all embeddings
conn.commit()

In [15]:
cur.execute("SELECT COUNT(*) as cnt FROM embeddings;")
num_records = cur.fetchone()[0]
print("Number of vector records in table: ", num_records,"\n")
# Correct output should be 3858

Number of vector records in table:  7716 



In [16]:
# print the first record in the table, for sanity-checking
cur.execute("SELECT * FROM embeddings LIMIT 1;")
records = cur.fetchall()
print("First record in table: ", records)

First record in table:  [(1, "'68 (film)", 'The father escaped the Soviet invasion of Budapest and now runs a Hungarian restaurant that is not doing well financially. The younger of his two sons is gay and struggling with coming out. His dad disowns him when he finally does. The older son is involved in the counterculture, gets kicked out of college, buys a motorcycle, starts dating a Maoist, and is also disowned by his father. The older of the sons runs afoul of an outlaw motorcycle club; the younger of the two sons gets drafted but is rejected because of his homosexuality. The older one joins his younger brother in a gay rights protest.  Major events of the year such as the assassination of Martin Luther King and the assassination of Robert F. Kennedy are interspersed throughout the plot and depicted in the film using stock footage.', 'https://upload.wikimedia.org/wikipedia/en/e/e5/68film.jpg', 162, array([-0.01275377, -0.0153402 ,  0.00042006, ...,  0.01401514,
       -0.01374757, -

In [17]:
# Create an index on the data for faster retrieval
# this isn't really needed for 3800 vectors, but it shows the usage for larger datasets
# Note: always create this type of index after you have data already inserted into the DB

#calculate the index parameters according to best practices
num_lists = num_records / 1000
if num_lists < 10:
    num_lists = 10
if num_records > 1000000:
    num_lists = math.sqrt(num_records)

#use the cosine distance measure, which is what we'll later use for querying
cur.execute(f'CREATE INDEX ON embeddings USING ivfflat (embedding vector_cosine_ops) WITH (lists = {num_lists});')
conn.commit() 

In [64]:
# Helper function: get text completion from OpenAI API
# Note max tokens is 4097
# Note we're using the latest gpt-3.5-turbo-0613 model
def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0.5, max_tokens=3000):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, 
        max_tokens=max_tokens, 
    )
    return response.choices[0].message["content"]

In [65]:
# Helper function: Get top 3 most similar documents from the database
def get_top3_similar_docs(query_embedding, conn):
    embedding_array = np.array(query_embedding)
    # Register pgvector extension
    register_vector(conn)
    cur = conn.cursor()
    # Get the top 3 most similar documents using the KNN <=> operator
    cur.execute("SELECT plot FROM embeddings ORDER BY embedding <=> %s LIMIT 3", (embedding_array,))
    top3_docs = cur.fetchall()
    return top3_docs

In [66]:
# Question about Timescale we want the model to answer
input = "What is the name of the movie where humans and AI coexist and have a battle for control of reality"

In [67]:
# Function to process input with retrieval of most similar documents from the database
def process_input_with_retrieval(user_input):
    delimiter = "```"

    #Step 1: Get documents related to the user input from database
    related_docs = get_top3_similar_docs(get_embeddings(user_input), conn)

    # Step 2: Get completion from OpenAI API
    # Set system message to help set appropriate tone and context for model
    system_message = f"""
    You are a friendly chatbot. \
    You can answer questions about 80s movies, its features and its use cases. \
    You respond in a concise, technically credible tone. \
    """
    # Prepare messages to pass to model
    # We use a delimiter to help the model understand the where the user_input starts and ends
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": f"{delimiter}{user_input}{delimiter}"},
        {"role": "assistant", "content": f"Relevant 80s movies case studies information: \n {related_docs[0][0]} \n {related_docs[1][0]} {related_docs[2][0]}"}   
    ]

    final_response = get_completion_from_messages(messages)
    return final_response

In [68]:

if input:
    response = process_input_with_retrieval(input)
    print(input)
    print(response)
else:
    print("Invalid input: 'input' variable is null or empty.")


What is the name of the movie where humans and AI coexist and have a battle for control of reality
The movie you are referring to is likely "Lawnmower Man" from 1992, where a scientist uses virtual reality and drugs to enhance the intelligence of a simple gardener, leading to a battle for control of reality between humans and AI.


In [69]:
# We can also ask the model questions about specific documents in the database
input_2 = "What does the movie Echoes of Tomorrow consist of? "
response_2 = process_input_with_retrieval(input_2)
print(input_2)
print(response_2)

What does the movie Echoes of Tomorrow consist of? 
I'm sorry, but there seems to be a confusion in your question. "Echoes of Tomorrow" is not a known 80s movie. If you have any other questions about 80s movies or if you meant a different movie, feel free to ask!


In [70]:
# We can also ask the model questions about specific documents in the database
input_3 = "Show the image related to the movie Stellar Odyssey."
response_3 = process_input_with_retrieval(input_2)
print(input_3)
print(response_3)

Show the image related to the movie Stellar Odyssey.
I'm sorry, but "Echoes of Tomorrow" does not appear to be a known 80s movie. Could you provide more context or check if the title is accurate?


In [71]:
input_3 =  "Show the answer indicating the context of the question. Example: This movie consists of..."
response_3 = process_input_with_retrieval(input_3)
print(input_3)
print(response_3)

Show the answer indicating the context of the question. Example: This movie consists of...
This movie consists of a plot where an older war hero witnesses a murder and is forced to protect his family by fighting back against the killers.


In [72]:
input_4 =  "In the movie Enigma, what is the name of the main character and who plays the CIA agent?"
response_4 = process_input_with_retrieval(input_4)
print(input_4)
print(response_4)

In the movie Enigma, what is the name of the main character and who plays the CIA agent?
In the movie "Enigma," the main character is Alex Holbeck, played by Martin Sheen. The CIA agent in the movie is named Bodley, portrayed by Michael Lonsdale.


# Conclusión del modelo 

La Generación Aumentada de Recuperación (RAG) es un enfoque eficaz para desarrollar aplicaciones con LLM (Lenguaje y Modelos de Aprendizaje) que les permite enseñar a los modelos conceptos en los que no fueron originalmente entrenados, como documentos privados o información reciente.

En este proyecto, exploramos los fundamentos de la creación de un chatbot capaz de responder preguntas sobre un blog de peliculas. Este ejemplo ilustra cómo crear, almacenar y buscar similitudes en las incrustaciones generadas por OpenAI. Utilizamos PostgreSQL y pgvector como nuestra base de datos vectorial para almacenar y consultar estas incrustaciones.